In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
DATA_DIR = './data/'
CANCER_DATA = 'cancer_incd_rate_2016_2020.csv'
AQI_DATA = lambda year: f'annual_aqi_by_county_{year}.csv'
US_COUNTIES = 'uscounties/uscounties.csv'

In [25]:
def get_uscounties():
    df = pd.read_csv(DATA_DIR + US_COUNTIES).rename({'county': 'County', 'county_fips': 'FIPS'}, axis=1)
    df["FIPS"] = pd.to_numeric(df["FIPS"], errors='coerce')
    df["FIPS"] = df["FIPS"].astype('Int64')
    return df
get_uscounties()

,County,county_ascii,county_full,FIPS,state_id,state_name,lat,lng,population
0,Los Angeles,Los Angeles,Los Angeles County,6037,CA,California,34.3219,-118.2247,9936690
1,Cook,Cook,Cook County,17031,IL,Illinois,41.8401,-87.8168,5225367
2,Harris,Harris,Harris County,48201,TX,Texas,29.8578,-95.3938,4726177
3,Maricopa,Maricopa,Maricopa County,4013,AZ,Arizona,33.3490,-112.4915,4430871
4,San Diego,San Diego,San Diego County,6073,CA,California,33.0343,-116.7351,3289701
...,...,...,...,...,...,...,...,...,...
3139,Blaine,Blaine,Blaine County,31009,NE,Nebraska,41.9128,-99.9768,384
3140,King,King,King County,48269,TX,Texas,33.6165,-100.2558,216
3141,Kenedy,Kenedy,Kenedy County,48261,TX,Texas,26.9285,-97.7017,116
3142,Loving,Loving,Loving County,48301,TX,Texas,31.8493,-103.5800,96


In [5]:
def replace_nan_variants(df: pd.DataFrame):
    # replace * and data not available variants with nan
    df = df.replace('data not available', np.nan)
    df = df.replace('data not available ', np.nan)
    df = df.replace(' data not available', np.nan)
    df = df.replace('*', np.nan)
    df = df.replace('* ', np.nan)
    df = df.replace(' *', np.nan)
    df = df.replace(' * ', np.nan)
    df = df.replace('3 or fewer', '3')
    return df
    

def convert_to_numeric(df: pd.DataFrame):
    df['Recent 5-Year Trend'] = pd.to_numeric(df['Recent 5-Year Trend'], errors='coerce')
    df['Incidence Rate per 100k'] = pd.to_numeric(df['Incidence Rate per 100k'], errors='coerce')
    df['Lower 95% Confidence Interval'] = pd.to_numeric(df['Lower 95% Confidence Interval'], errors='coerce')
    df['Upper 95% Confidence Interval'] = pd.to_numeric(df['Upper 95% Confidence Interval'], errors='coerce')
    df['Average Annual Count'] = pd.to_numeric(df['Average Annual Count'], errors='coerce')
    df['FIPS'] = pd.to_numeric(df['FIPS'], errors='coerce')
    df["FIPS"] = df["FIPS"].astype('Int64')
    return df

def merge_locational_data(df: pd.DataFrame):
    coords_df = get_uscounties()
    df = df.merge(
        coords_df, on='FIPS'
    ).drop(
        columns=['County_x', 'state_id', 'county_full','county_ascii', 'State']
    ).rename(
        {'County_y': 'County', 'state_name': 'State', 'lat': 'Lat', 'lng': 'Lon', 'population': 'Population'}, 
        axis=1
    )
    return df

def get_cancer_data():
    df = pd.read_csv(DATA_DIR + CANCER_DATA, skiprows=8, skipfooter=31, engine='python')
    df.insert(0, 'State', df['County'].apply(lambda x: x.split(', ')[-1][:-3]))
    df['County'] = df['County'].apply(lambda x: x.split(', ')[0])
    df = df.iloc[1:].rename({
        'Age-Adjusted Incidence Rate([rate note]) - cases per 100,000': 'Incidence Rate per 100k', 
        'Recent 5-Year Trend ([trend note]) in Incidence Rates': 'Recent 5-Year Trend',
        ' FIPS': 'FIPS'
    }, axis=1)

    df = df.pipe(replace_nan_variants).pipe(convert_to_numeric).pipe(merge_locational_data)
    df = df[df['Recent 5-Year Trend'].notna()]
    return df
get_cancer_data()

,FIPS,Incidence Rate per 100k,Lower 95% Confidence Interval,Upper 95% Confidence Interval,CI*Rank([rank note]),Lower CI (CI*Rank),Upper CI (CI*Rank),Average Annual Count,Recent Trend,Recent 5-Year Trend,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1,County,State,Lat,Lon,Population
0,12125,1237.4,1165.6,1312.8,N/A,1,1,237.0,stable,0.6,-0.5,1.9,Union,Florida,30.0439,-82.3714,15524
1,19147,658.1,591.1,731.1,N/A,1,6,82.0,rising,4.8,0.2,15.4,Palo Alto,Iowa,43.0821,-94.6781,8938
2,30103,652.2,401.0,1007.4,N/A,1,55,7.0,stable,-1.1,-5.6,3.3,Treasure,Montana,46.2115,-107.2716,680
3,48373,633.6,604.6,663.7,N/A,1,4,425.0,rising,2.2,1.2,4.2,Polk,Texas,30.7927,-94.8300,50536
4,21071,616.8,584.3,650.7,N/A,1,19,295.0,stable,1.5,-1.8,5.3,Floyd,Kentucky,37.5571,-82.7457,35780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2677,31091,246.3,144.1,431.4,N/A,32,87,4.0,stable,-3.2,-7.3,1.2,Hooker,Nebraska,41.9161,-101.1353,659
2678,53075,244.8,223.0,268.1,N/A,37,39,99.0,falling,-3.7,-5.2,-2.3,Whitman,Washington,46.9012,-117.5230,47141
2679,26083,240.0,171.7,336.7,N/A,74,83,12.0,stable,-1.2,-4.9,2.7,Keweenaw,Michigan,47.6279,-88.4346,2088
2680,8079,235.4,139.5,412.7,N/A,3,62,4.0,stable,-0.3,-7.8,7.8,Mineral,Colorado,37.6689,-106.9241,794


In [28]:
def merge_locational_data(df: pd.DataFrame):
    coords_df = get_uscounties()
    df = df.merge(
        coords_df, on=['County']
    ).drop_duplicates(subset=['County'],keep='first')
    # .drop(
    #     columns=['County_x', 'state_id', 'county_full','county_ascii', 'State']
    # ).rename(
    #     {'County_y': 'County', 'state_name': 'State', 'lat': 'Lat', 'lng': 'Lon', 'population': 'Population'}, 
    #     axis=1
    # )
    return df

# compile aqi data into 5 year avaerage
def get_aqi_data():
    dfs = []
    
    for year in range(2016, 2021):
        aqi = pd.read_csv(DATA_DIR + AQI_DATA(year))
        dfs.append(aqi)
    
    annual_aqi_2016_2020 = pd.concat(dfs)
    annual_aqi_2016_2020 = annual_aqi_2016_2020.groupby(['State', 'County']).agg(
        {
            'Good Days': 'sum', 
            'Moderate Days': 'sum', 
            'Unhealthy for Sensitive Groups Days': 'sum',
            'Unhealthy Days': 'sum',
            'Very Unhealthy Days': 'sum',
            'Hazardous Days': 'sum',
            'Max AQI': 'max',
            'Median AQI': 'mean',
            'Days CO': 'sum',
            'Days NO2': 'sum',
            'Days Ozone': 'sum',
            'Days PM2.5': 'sum',
            'Days PM10': 'sum',
        }
    ).reset_index()

    annual_aqi_2016_2020 = annual_aqi_2016_2020.pipe(merge_locational_data)
    
    return annual_aqi_2016_2020

get_aqi_data()

,State,County,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,Median AQI,...,Days PM2.5,Days PM10,county_ascii,county_full,FIPS,state_id,state_name,lat,lng,population
0,Alabama,Baldwin,1220,138,1,0,0,0,108,36.20,...,300,0,Baldwin,Baldwin County,1003,AL,Alabama,30.7277,-87.7226,233420
2,Alabama,Clay,512,47,0,0,0,0,86,28.60,...,559,0,Clay,Clay County,29047,MO,Missouri,39.3105,-94.4209,253085
38,Alabama,Colbert,1026,78,1,0,0,0,115,36.75,...,231,0,Colbert,Colbert County,1033,AL,Alabama,34.7005,-87.8049,57270
39,Alabama,DeKalb,1623,158,1,0,0,0,119,37.80,...,167,0,DeKalb,DeKalb County,13089,GA,Georgia,33.7716,-84.2264,761209
57,Alabama,Elmore,936,54,0,0,0,0,100,35.60,...,0,0,Elmore,Elmore County,1051,AL,Alabama,32.5966,-86.1492,87694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4827,Wyoming,Sublette,1321,481,21,4,0,0,200,45.80,...,70,1,Sublette,Sublette County,56035,WY,Wyoming,42.7669,-109.9147,8801
4828,Wyoming,Sweetwater,1274,525,19,7,1,1,1250,46.00,...,28,271,Sweetwater,Sweetwater County,56037,WY,Wyoming,41.6595,-108.8796,42079
4829,Wyoming,Teton,1560,253,9,5,0,0,161,42.80,...,104,3,Teton,Teton County,56039,WY,Wyoming,43.9346,-110.5898,23346
4832,Wyoming,Uinta,1576,245,5,0,0,1,696,42.80,...,0,173,Uinta,Uinta County,56041,WY,Wyoming,41.2876,-110.5476,20546


# Cancer EDA

In [7]:
cancer_df = get_cancer_data()
fig = px.scatter(
    cancer_df, 
    x='Incidence Rate per 100k', 
    y='Recent 5-Year Trend', 
    hover_name='County', 
)
fig.show()

In [8]:
fig = px.histogram(
    cancer_df, 
    x='Incidence Rate per 100k', 
    nbins=100, 
    title='Cancer Incidence Rate per 100k Distribution'
)
fig.show()

In [9]:
fig = px.histogram(
    cancer_df, 
    x='Recent 5-Year Trend', 
    nbins=100, 
    title='Cancer Incidence Rate per 100k Distribution'
)
fig.show()

In [10]:
# normalize trend data
min_trend_value = cancer_df['Recent 5-Year Trend'].min()
max_trend_value = cancer_df['Recent 5-Year Trend'].max()
print(min_trend_value, max_trend_value)
cancer_df['Recent 5-Year Trend'] = cancer_df['Recent 5-Year Trend'].apply(lambda x: (x-min_trend_value)/(max_trend_value-min_trend_value))
cancer_df
# geo map of cancer data
fig = px.scatter_geo(
    cancer_df, 
    lat='Lat', 
    lon='Lon', 
    hover_name='County', 
    hover_data=['Incidence Rate per 100k', 'Recent 5-Year Trend', 'Population'],
    size='Average Annual Count', 
    title='Average Annual Count',
    color='Recent 5-Year Trend',
    projection='albers usa',
    height=500,
    width=1000
)
fig.update_layout(
    title_text='Average Annual Count',
    title_x=0.5
)
fig.show()

-26.9 21.4


In [11]:
cancer_df = get_cancer_data()
fig = px.scatter_geo(
    cancer_df, 
    lat='Lat', 
    lon='Lon', 
    hover_name='County', 
    hover_data=['Incidence Rate per 100k', 'Recent 5-Year Trend', 'Population'],
    color='Recent 5-Year Trend', 
    projection='albers usa',
    width=1000,
    height=500
)
fig.update_layout(
    title_text='Recent 5-Year Trend',
    title_x=0.5
)

fig.show()

In [12]:
cancer_df = get_cancer_data()
avg_cancer_df = cancer_df.groupby('State').agg(
    {
        'Incidence Rate per 100k': 'mean',
        'Recent 5-Year Trend': 'mean'
    }
).reset_index()
fig = px.bar(
    avg_cancer_df, 
    x='State', 
    y='Recent 5-Year Trend', 
    title='Recent 5-Year Trend by State'
)
fig.show()

In [13]:
fig = px.bar(
    avg_cancer_df,
    x='State', 
    y='Incidence Rate per 100k', 
    title='Incidence Rate per 100k v State',
    color='Recent 5-Year Trend'
)
fig.update_layout(
    title_x=0.5
)
fig.show()

# AQI EDA

In [20]:
aqi_df = get_aqi_data()
fig = px.scatter_geo(
    aqi_df, 
    lat='Lat', 
    lon='Lon', 
    hover_name='County', 
    hover_data=['Median AQI', 'Max AQI'],
    color='Max AQI', 
    projection='albers usa',
    width=1000,
    height=500
)

ValueError: Value of 'lat' is not the name of a column in 'data_frame'. Expected one of ['State', 'County', 'Good Days', 'Moderate Days', 'Unhealthy for Sensitive Groups Days', 'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI', 'Median AQI', 'Days CO', 'Days NO2', 'Days Ozone', 'Days PM2.5', 'Days PM10'] but received: Lat